In [ ]:
from os import environ
import urllib
import json

from sqlalchemy import create_engine, inspect
from sqlalchemy import Table, Column, ForeignKey, Integer, String, Float, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref
from sqlalchemy.orm import sessionmaker, selectinload
from sqlalchemy.sql import union, select, and_, or_, not_, text
from sqlalchemy.sql import bindparam
from sqlalchemy.sql.functions import coalesce


# Connect

In [ ]:
driver = environ.get('SQL_DRIVER', '{ODBC Driver 17 for SQL Server}')
host = environ.get('SQL_HOST', 'sql-fabulous')
db = environ.get('SQL_DB', 'ScratchDB')
user = environ.get('SQL_USER', 'sa')
pw = environ.get('SQL_PASSWORD', 'HelloWorld1')
con_str = f'DRIVER={driver};SERVER={host};DATABASE={db};UID={user};PWD={pw}'

params = urllib.parse.quote_plus(con_str)
sql_engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}"
                           , echo=True) # echo's emitted sql

# Define Schema

In [ ]:
Base = declarative_base()

class Survey(Base):
    __tablename__ = 'DirectionalSurvey2'

    ID = Column(Integer, autoincrement=True, primary_key=True)
    API = Column(String(32), nullable=True)
    WKID = Column(String(32), nullable=True)
    FIPS = Column(String(4), nullable=True)
    STATUS_CODE = Column(String(1), nullable=False)

    def __repr__(self):
        return f"Survey(ID={self.ID}, API={self.API}, WKID={self.WKID}, Points={self.stations})"

class SurveyReport(Base):
    __tablename__ = 'SurveyReport2'

    ID = Column(Integer, autoincrement=True, primary_key=True)
    DirectionalSurveyId = Column(Integer, ForeignKey('DirectionalSurvey2.ID'), nullable=False)
    Azimuth = Column(Float, nullable=True)
    MD = Column(Float, nullable=True)
    Inclination = Column(Float, nullable=True)
    STATUS_CODE = Column(String(1), nullable=False)
    survey = relationship(Survey, backref=backref('stations', uselist=True))

    def __repr__(self):
        return f"Report(ID={self.ID}, FK={self.DirectionalSurveyId}, STATUS={self.STATUS_CODE})"

# Set mapped tables to local vars so that full SQL metadata is available.
surveys = Survey.__table__
points = SurveyReport.__table__

# Bind Engine to Schema and Create Session, Connection

In [ ]:
Base.metadata.bind = sql_engine
DBSession = sessionmaker(bind=sql_engine)
session = DBSession()
conn = sql_engine.connect()

# Create Schema

In [ ]:
Base.metadata.drop_all()
Base.metadata.create_all(sql_engine, checkfirst=True)

# Create Bean

## Atomic Commits (parent/child)

In [ ]:
try:
    survey1 = Survey(API='API1', WKID='WKID1', FIPS = '0001', STATUS_CODE='C')

    survey_report1 = SurveyReport(DirectionalSurveyId=survey1.ID, Azimuth=1, MD=1, Inclination=1, STATUS_CODE='C')
    survey_report2 = SurveyReport(DirectionalSurveyId=survey1.ID, Azimuth=2, MD=2, Inclination=1, STATUS_CODE='C')
    survey_report3 = SurveyReport(DirectionalSurveyId=survey1.ID, Azimuth=3, MD=3, Inclination=1, STATUS_CODE='C')

    survey1.stations.append(survey_report1)
    survey1.stations.append(survey_report2)
    survey1.stations.append(survey_report3)

    session.add(survey1)
    
    survey2 = Survey(API='API2', WKID='WKID2', FIPS = '0002', STATUS_CODE='C')

    survey_report2b = SurveyReport(DirectionalSurveyId=survey2.ID, Azimuth=2, MD=2, Inclination=2, STATUS_CODE='N')
    survey_report3b = SurveyReport(DirectionalSurveyId=survey2.ID, Azimuth=3, MD=3, Inclination=2, STATUS_CODE='N')
    survey_report4b = SurveyReport(DirectionalSurveyId=survey2.ID, Azimuth=4, MD=4, Inclination=2, STATUS_CODE='N')

    survey2.stations = [survey_report2b, survey_report3b, survey_report4b]

    session.add(survey2)
    
    survey3 = Survey(API='API3', WKID='WKID3', FIPS = '0003', STATUS_CODE='N')

    survey_report2c = SurveyReport(DirectionalSurveyId=survey3.ID, Azimuth=2, MD=2, Inclination=2, STATUS_CODE='N')
    survey_report3c = SurveyReport(DirectionalSurveyId=survey3.ID, Azimuth=3, MD=3, Inclination=2, STATUS_CODE='N')
    survey_report4c = SurveyReport(DirectionalSurveyId=survey3.ID, Azimuth=4, MD=4, Inclination=2, STATUS_CODE='N')

    survey3.stations.extend([survey_report2c, survey_report3c, survey_report4c])
    
    session.add_all([survey3])
    session.commit()

#     session.add_all([survey_report1])
#     session.commit()

except Exception as e:
    print(e)
    session.rollback()

# Select Using SQL ORM

In [ ]:
my_surveys = session.query(Survey) \
                .options(selectinload(Survey.stations))\
                .filter(Survey.STATUS_CODE.in_(['N', 'C']))\
                .filter(Survey.stations.any())\
                .all()

# List of Survey
try:
    for s in my_surveys:
        print(f"S: {s.ID}, {s.stations}")
        in_state = inspect(s)
        
        for ar in in_state.attrs:
            print(f"lv: {ar.loaded_value}, v:{ar.value}")
        #print(s.__table__.columns)
        
#         for sr in s.stations:
#             print(f"SR: {sr.ID}, {sr.STATUS_CODE}")
#             print(type(sr))
#             print(sr.__table__.columns)
            
except Exception as e:
    print(f"{e}")


# Select Using SQL Core

In [ ]:
stmt = select([surveys, points.c.ID.label('ReportId')])\
        .select_from(surveys.join(points))\
        .where(
            and_(
                surveys.c.API.like('API%'),
                points.c.STATUS_CODE.in_(['C', 'N']),
                points.c.Azimuth >= 6
            )
        )

cursor = conn.execute(stmt)

# List of RowProxy
for r in cursor:
    print(f"{type(r)}, {r.keys()}, {r[surveys.c.ID]}, {r['ReportId']}")
    print(json.dumps(r.items()))
    
cursor.close()

In [ ]:
stmt2 = select([surveys, points.c.Azimuth])\
        .select_from(surveys.join(points))\
        .where(
            and_(
                surveys.c.API.like('API%'),
                points.c.STATUS_CODE.in_(['C', 'N']),
                points.c.Azimuth >= 6
            )
        )

try:
    cursor = conn.execute(stmt2)

    # List of RowProxy
    for r in cursor:
        print(f"K:{r.keys()}, KV: {r.items()}")
        print(json.dumps(r.items()))
        
except Exception as e:
    print(f"{e}")
finally:
    cursor.close()